In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/Shareddrives/My\ paper

/content/drive/Shareddrives/My paper


In [ ]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from collections import Counter

In [ ]:
member = pd.read_csv('dataset/91APP_MemberData.csv')

In [ ]:
order = pd.read_csv('dataset/91APP_OrderData.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## EDA: Member data & Order Data

In [ ]:
print(len(member['MemberID'].unique()), len(order['MemberID'].unique()))

377133 235493


In [ ]:
order['TradesDateTime'] = pd.to_datetime(order['TradesDateTime'])
order = order.sort_values(by='TradesDateTime')
order = order[(order['Status']=='Finish')|(order['Status']=='Return')]

In [ ]:
order['TradeYear'] = order['TradesDateTime'].dt.year
order['TradeMonth'] = order['TradesDateTime'].dt.month

In [ ]:
member['RegisterDateTime'] = pd.to_datetime(member['RegisterDateTime'])
member = member.sort_values(by='RegisterDateTime')
member = member.drop_duplicates(subset=['MemberID'], keep=False)

In [ ]:
len(member)

377132

In [ ]:
len(member[member['RegisterDateTime']<'2020-05-01']['MemberID'].unique())

313647

In [ ]:
order_2019 = order[(order['TradesDateTime']>='2019-05-01')&(order['TradesDateTime']<'2020-05-01')]
order_2020 = order[(order['TradesDateTime']>='2020-05-01')&(order['TradesDateTime']<'2021-05-01')]

In [ ]:
print(len(order_2019['MemberID'].unique()), len(order_2020['MemberID'].unique()))
print(len(order_2020[order_2020['MemberID'].isin(order_2019['MemberID'])]['MemberID'].unique()))
print(len(order_2019[order_2019['MemberID'].isin(member[member['RegisterDateTime']<'2020-05-01']['MemberID'])]['MemberID'].unique()))
print(len(order_2020[order_2020['MemberID'].isin(member[member['RegisterDateTime']<'2020-05-01']['MemberID'])]['MemberID'].unique()))

59417 71460
16797
59416
33194


In [ ]:
member['RegisterYear'] = member['RegisterDateTime'].dt.year

In [ ]:
memberIDs = list(member[member['RegisterDateTime']<'2020-05-01']['MemberID'].unique())

In [ ]:
print('2020.05.01前註冊的會員:', len(memberIDs), '人')

2020.05.01前註冊的會員: 313647 人


## 抓取有行為資料的 MemberID

### Step 1: 處理行為資料

In [ ]:
# 十大事件
event_list = ['trafficSource', 'userRegisteration', 'mainPageView', 'productPageView', 'categoryPageView', 
              'activityPageView', 'search', 'addToCart', 'checkout', 'purchase']

bh_list = ['traffic', 'register', 'viewmainpage', 'viewproduct', 'viewcategory', 'viewactivity', 'search', 'add', 'checkout', 'purchase']

In [ ]:
# 抓出期間內的所有行為資料
def merge_data(events, dates):
    i = 1
    paths = []
    for date_str in dates:
        for event in events:
            path = './dataset/91APP_BehaviorData/' + event + '_' + date_str + '.csv'
            paths.append(path)
  
    dfs = [pd.read_csv(path, low_memory=False, encoding = "utf-8") for path in paths]
    df = pd.concat(dfs)     

    df['DateTime'] = pd.to_datetime(df.ht, unit='ms')

    df = df.sort_values(by = ['did', 'DateTime'])
    df = df[['dc', 'did', 'uid', 'DateTime', 'bh', 'cs', 'cm', 'cn', 'pr', 'pr1id', 'pr1pr', 'pr1qt', 'cu', 'tr', 'ti']]
    
    return df

In [ ]:
# 合併每個月的行為資料
month_start = [x.strftime('%F')for x in pd.date_range("2019-05-01", freq="MS", periods=12)]
month_end = [x.strftime('%F')for x in pd.date_range("2019-05-01", freq="M", periods=12)]

for m in range(len(month_start)):
  print(m) # month index
  date_range = pd.date_range(month_start[m], month_end[m])
  date_list = [x.strftime('%F')for x in date_range]

  month_df = merge_data(event_list, date_list)
  file_name = 'Month ' + str(m) + '.csv'
  month_df.to_csv(file_name, index=False, encoding = 'utf-8')

In [ ]:
months = range(1,13)
paths = []
for m in months:
  path = './Customer Value Prediction/TABLES/Behavior Data/Month '+ str(m) + '.csv'
  paths.append(path) 

behavior_data = [pd.read_csv(path, low_memory=False, encoding = "utf-8")[['dc', 'did', 'uid', 'DateTime', 'bh']] for path in paths]

### Step 2: 轉換 UID 欄位

In [ ]:
member_data = member[member['MemberID'].isin(memberIDs)]

In [ ]:
def add_uid_col(data):
  df = data.copy()
  df['uid'] = df['MemberID'].str.replace('+', '%2B')
  df['uid'] = df['uid'].str.replace('/', '%2F')
  df['uid'] = df['uid'].str.replace('=', '%3D')

  return df['uid']

In [ ]:
member_data['uid'] = add_uid_col(member_data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
uids = list(member_data['uid'].unique())

In [ ]:
print(len(memberIDs), len(uids))

313647 313647


### Step 3: 取得 MemberID, uid, did 對照表

In [ ]:
dfs = []
for m in range(len(behavior_data)):
  df = behavior_data[m][(behavior_data[m]['uid'].isin(uids))&(behavior_data[m]['did'].notna())].drop_duplicates(subset=['did', 'uid'], keep='last')[['did', 'uid']]

  dfs.append(df)

uid_df = pd.concat(dfs).drop_duplicates(subset=['did', 'uid'], keep='last')[['did', 'uid']]

In [ ]:
len(uid_df['uid'].unique())

120080

In [ ]:
id_df = pd.merge(uid_df.drop_duplicates(subset=['uid'], keep='last')[['uid']], member_data[['MemberID', 'uid']], on='uid', how='left')

In [ ]:
memberIDs = list(id_df['MemberID'].unique())
uids = list(id_df['uid'].unique())
print(len(memberIDs), len(uids))

120080 120080


In [ ]:
print(len(order_2020[order_2020['MemberID'].isin(memberIDs)]['MemberID'].unique()))

24169


### Step 4: 合併行為資料


In [ ]:
order_data = order[order['MemberID'].isin(memberIDs)]
# only consider finish and return order
order_data = order_data[(order_data['Status'] == 'Finish') | (order_data['Status'] == 'Return')]

In [ ]:
print('-------------------Start to process session times-------------------')
id_dfs = []

print('Get id table')
print('uid len:', len(uids))
for m in list(range(12)):
  df = behavior_data[m]
  id_dfs.append(df.loc[df['uid'].isin(uids)][['did', 'uid']]) 
id_table = pd.concat(id_dfs).drop_duplicates(subset=['did'], keep='last')
print('id_table len:', len(id_table))

-------------------Start to process session times-------------------
Get id table
uid len: 120080
id_table len: 171045


In [ ]:
print('-------------------Get behavior data of members----------------------')
dfs = []
for m in list(range(12)):
  df = behavior_data[m]
  dfs.append(df.loc[(df['did'].isin(id_table['did']))])

behaviors = pd.concat(dfs)
print('Process ids of member')
# 一個 uid 有多個 did 的問題
fill_uid = pd.merge(behaviors[['did']], id_table, how='left', on='did').rename(columns = {'uid': 'uid_filled'}, inplace = False)
behaviors['uid_filled'] = list(fill_uid['uid_filled'])
behaviors = behaviors.sort_values(by = ['uid_filled', 'DateTime'])

-------------------Get behavior data of members----------------------
Process ids of member


In [ ]:
behaviors.to_csv('all behaviors data.csv')